In [34]:
%%pyspark

df = spark.read.load('abfss://capture@spkaccelerjqvse6bhhchxi.dfs.core.windows.net/SeattlePublicLibrary/Checkouts_By_Title__Physical_Items_.csv', format='csv'
## If header exists uncomment line below
, header=True
)
display(df.limit(10))

StatementMeta(synasp1, 3, 34, Finished, Available)

SynapseWidget(Synapse.DataFrame, 186aadd0-62df-4df3-97d6-0f278686b59e)

In [35]:
%%pyspark

# Show Schema
df.printSchema()

StatementMeta(synasp1, 3, 35, Finished, Available)

root
 |-- ID: string (nullable = true)
 |-- CheckoutYear: string (nullable = true)
 |-- BibNumber: string (nullable = true)
 |-- ItemBarcode: string (nullable = true)
 |-- ItemType: string (nullable = true)
 |-- Collection: string (nullable = true)
 |-- CallNumber: string (nullable = true)
 |-- ItemTitle: string (nullable = true)
 |-- Subjects: string (nullable = true)
 |-- CheckoutDateTime: string (nullable = true)

In [36]:
%%pyspark

from pyspark.sql import SparkSession
from pyspark.sql.types import *

# Primary storage info
capture_account_name = 'spkaccelerjqvse6bhhchxi' # fill in your primary account name
capture_container_name = 'capture' # fill in your container name
capture_relative_path = 'SeattlePublicLibrary/Checkouts_By_Title__Physical_Items_.csv' # fill in your relative folder path

capture_adls_path = 'abfss://%s@%s.dfs.core.windows.net/%s' % (capture_container_name, capture_account_name, capture_relative_path)
print('Primary storage account path: ' + capture_adls_path)

StatementMeta(synasp1, 3, 36, Finished, Available)

Primary storage account path: abfss://capture@spkaccelerjqvse6bhhchxi.dfs.core.windows.net/SeattlePublicLibrary/Checkouts_By_Title__Physical_Items_.csv

In [37]:
%%pyspark

from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType, DateType, TimestampType
csvSchema = StructType([
  StructField('id', StringType(), True), 
  StructField('checkout_year', IntegerType(), True),
  StructField('bib_number', IntegerType(), True),  
  StructField('item_barcode', StringType(), True),
  StructField('item_type', StringType(), True),
  StructField('collection', StringType(), True),
  StructField('call_number', StringType(), True),
  StructField('item_title', StringType(), True),
  StructField('subjects', StringType(), True),
  StructField('CheckoutDateTime', StringType(), True)      
])

CheckByTPI_capture_df = spark.read.format('csv').option('header', 'True').schema(csvSchema).load(capture_adls_path)

display(CheckByTPI_capture_df.limit(10))



StatementMeta(synasp1, 3, 37, Finished, Available)

SynapseWidget(Synapse.DataFrame, 44c01599-a957-4c76-9068-e6917ecf0b59)

In [38]:
%%pyspark

from pyspark.sql.functions import to_date, to_timestamp, col, date_format, current_timestamp
df_final = (CheckByTPI_capture_df.withColumn("checkout_date_time", to_timestamp(col("CheckoutDateTime"),"M/d/yyyy h:m:s a")).drop("CheckoutDateTime")
                                 .withColumn('loadDate', date_format(current_timestamp(), 'M/d/y H:m:s a'))
                                 .withColumn("load_date", to_timestamp(col("loadDate"),"M/d/y H:m:s a")).drop("loadDate")
) 

StatementMeta(synasp1, 3, 38, Finished, Available)

In [39]:
%%pyspark

# Show Schema
df_final.printSchema()

display(df_final.limit(10))


StatementMeta(synasp1, 3, 39, Finished, Available)

root
 |-- id: string (nullable = true)
 |-- checkout_year: integer (nullable = true)
 |-- bib_number: integer (nullable = true)
 |-- item_barcode: string (nullable = true)
 |-- item_type: string (nullable = true)
 |-- collection: string (nullable = true)
 |-- call_number: string (nullable = true)
 |-- item_title: string (nullable = true)
 |-- subjects: string (nullable = true)
 |-- checkout_date_time: timestamp (nullable = true)
 |-- load_date: timestamp (nullable = true)



SynapseWidget(Synapse.DataFrame, 2509ed5b-8af2-42e4-97fb-f47c549b223f)

In [40]:
%%pyspark

from pyspark.sql import SparkSession
from pyspark.sql.types import *

# Primary storage info
compose_account_name = 'spkaccelerjqvse6bhhchxi' # fill in your primary account name
compose_container_name = 'compose' # fill in your container name
compose_relative_path = 'SeattlePublicLibrary/CheckoutsByTitlePhysicalItems/' # fill in your relative folder path

compose_adls_path = 'abfss://%s@%s.dfs.core.windows.net/%s' % (compose_container_name, compose_account_name, compose_relative_path)
print('Primary storage account path: ' + compose_adls_path)

StatementMeta(synasp1, 3, 40, Finished, Available)

Primary storage account path: abfss://compose@spkaccelerjqvse6bhhchxi.dfs.core.windows.net/SeattlePublicLibrary/CheckoutsByTitlePhysicalItems/

In [41]:
%%pyspark

compose_parquet_path = compose_adls_path + 'checkouts.parquet'

print('parquet file path: ' + compose_parquet_path)

StatementMeta(synasp1, 3, 41, Finished, Available)

parquet file path: abfss://compose@spkaccelerjqvse6bhhchxi.dfs.core.windows.net/SeattlePublicLibrary/CheckoutsByTitlePhysicalItems/checkouts.parquet

In [42]:
%%pyspark

df_final.write.parquet(compose_parquet_path, mode = 'overwrite')

StatementMeta(synasp1, 3, 42, Finished, Available)

In [43]:
%%sql

-- Create database SeattlePublicLibrary only if database with same name does not exist
CREATE DATABASE IF NOT EXISTS SeattlePublicLibrary

StatementMeta(synasp1, 3, 43, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

In [44]:
%%sql

-- Create table CheckoutsByTitlePhysicalItemsschemafinal only if table with same name does not exist
CREATE TABLE IF NOT EXISTS SeattlePublicLibrary.checkouts_by_title_physical_items
(id STRING
,checkout_year INT
,bib_number INT
,item_barcode STRING
,item_type STRING
,collection STRING
,call_number STRING
,item_title STRING
,subjects STRING
,checkout_date_time TIMESTAMP
,load_date TIMESTAMP
)
USING PARQUET OPTIONS (path 'abfss://compose@spkaccelerjqvse6bhhchxi.dfs.core.windows.net/SeattlePublicLibrary/CheckoutsByTitlePhysicalItems/checkouts.parquet')

StatementMeta(synasp1, 3, 44, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>